# Pregunta 2: RSA

## Test de primalidad de Rabin-Miller

In [1]:
def rabin_miller(n: int, k: int) -> bool:
    """
    Determines if a number is prime using Rabin-Miller primality test

    Args:
        n (int): number >= 1 to test for primality
        k (int): number >= 1, error threshold parameter

    Returns:
        bool: True if n is prime, False otherwise (with error probability <= 2**(-k))
    """
    return True


def generar_primo(l: int) -> int:
    """
    Generates a random prime number with at least l digits.

    Args:
        l (int): number >= 1 minimum number of digits of prime number

    Returns:
        int: Prime number with at least l digits. Error probability must be <= 2**(-100)
    """
    current_test = 10**l
    while not rabin_muller(current_test, 100):
        current_test += 1
    return current_test


## Algoritmo extendido de euclides

In [4]:
def alg_ext_euclides(a: int, b: int) -> tuple[int, int, int]:
    """
    Extended euclidean algorithm

    Args:
        a (int): number > 0
        b (int): a >= b >=

    Returns:
        tuple[int, int, int]: (GCD(a,b), s, t) greatest common divisor GCD(a, b) of a and b, and
            integers s and t: MCD(a, b) = s*a + t*b
    """
    return 1


## Exponenciación rápida

In [ ]:
def exp_mod(a: int, b: int, n: int) -> int:
    """
    Modular exponentation algorithm

    Args:
        a (int): number >= 0
        b (int): number >= 0
        n (int): number > 0

    Returns:
        int: a**b mod n
    """


## Inverso  modular

In [ ]:
def inverso(a: int, n: int) -> int:
    """
    Modular inverse

    Args:
        a (int): number >= 1
        n (int): number >= 2, relative prime of a

    Returns:
        int: modular inverse of a in mod n    
    """


## Generar clave 

In [ ]:
def generar_clave(l: int):
    """
    Build a public and private key of specific length.

    Args:
        l (int): length of keys to generate

    Saves private key (d, N) and public key (e, N) in private_key.txt and public_key.txt
    respectively, with the following format:
        d
        N
    """


## Encryption and decryption algorithms

In [ ]:
def enc(m: int) -> int:
    """
    Encrypts a message m with public key stored in public_key.txt

    Args:
        m (int): 0 <= m <= N-1, message to encrypt

    Returns:
        int: m encrypted with public key (e, N)
    
    """


def dec(m: int) -> int:
    """
    Decrypts a message m with private key stored in private_key.txt

    Args:
        m (int): 0 <= m <= N-1, message to decrypt

    Returns:
        int: m decrypted with private key (d, N)
    
    """
